In [ ]:
from neo4j import GraphDatabase, basic_auth
import pandas as pd


driver = GraphDatabase.driver(
  "bolt://18.212.185.199:7687",
  auth=basic_auth("neo4j", "companies-superintendent-chit"))

def read_cypher(query):
      with driver.session(database="neo4j") as session:
        results = session.run(query).data()
        df = pd.DataFrame(results)
        col_names = df.columns.values.tolist()
        for c in col_names:
          if type(df[c].iloc[0]) is dict:
            df = pd.concat([df, df[c].apply(pd.Series)], axis=1)
            df.drop(columns=c, inplace=True)
          else:
            df1 = df.pop(c)
            df[c] = df1
          return df

1. i nomi delle squadre che hanno partecipano al torneo nel 2015

In [ ]:
query = '''
MATCH (a:Team)-[p:PARTICIPATED_IN]-(t:Tournament) where t.year = 2015 RETURN a
'''

read_cypher(query)

2. i nomi delle squadre che hanno partecipato al torneo nel 2015 (farlo in un altro modo)

In [ ]:
query = ''' 

MATCH (a:Team)-[p:PARTICIPATED_IN]-(t:Tournament {year: 2015}) RETURN a
'''
read_cypher(query)

3. il numero di partecipazioni di ogni nazione

In [ ]:
query = ''' 

MATCH (a:Team)-[p:PARTICIPATED_IN]-(t:Tournament)
RETURN a as Nazione, count(p) as totale
'''
read_cypher(query)

4. gli allenatori dell'Italia

In [ ]:
query = ''' 

MATCH (p:Person)-[:COACH_FOR]-(s:Squad)
WHERE s.id STARTS WITH 'Italy'
RETURN p, s.id
'''
read_cypher(query)

5. le giocatrici dell'Italia che hanno segnato e in quale partita hanno segnato (data)

In [ ]:
query = ''' 

MATCH (m:Match)-[:SCORED_GOAL]-(p:Person)-[:REPRESENTS]-(t:Team{name:'Italy'})
RETURN p.name, m.date
'''
read_cypher(query)

6. il numero di gol segnato dalle giocatrici italiane in totale

In [ ]:
query = ''' 

MATCH (m:Match)-[g:SCORED_GOAL]-(p:Person)-[:REPRESENTS]-(t:Team{name:'Italy'})
RETURN count(g)

'''
read_cypher(query)

7. le giocatrici che hanno partecipato alla partita del 9 giugno 2019

In [ ]:
query = ''' 

MATCH (p:Person)-[:PLAYED_IN]-(m:Match)
WHERE m.date = date('2019-06-09')
RETURN p.name, m.date ORDER BY p.name

'''
read_cypher(query)

8. le giocatrici che hanno segnato nella partita del 9 giugno 2019

In [ ]:
query = ''' 

MATCH (p:Person)-[:SCORED_GOAL]-(m:Match)
WHERE m.date = date('2019-06-09')
RETURN p.name, m.date ORDER BY p.name

'''
read_cypher(query)

9. le giocatrici che non hanno segnato nelal partita del 9 giugno 2019

In [ ]:
query = ''' 

MATCH (p:Person)-[:PLAYED_IN]-(m:Match)
WHERE m.date = date('2019-06-09') 
AND NOT exists ((p)-[:SCORED_GOAL]-(m))
RETURN p.name

'''
read_cypher(query)